<a href="https://colab.research.google.com/github/Shahabshms/Numerical_Methods_for_ML_and_AI_Solution_2/blob/main/_4301_HW2_Q2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Problem 2: Projections onto Convex Hulls** 
For this problem, you will implement the Frank-Wolfe algorithm using scipy.linprog in the Python scipy package to help you solve the per-iteration subproblems. Recall that, given $x^{(1)}, \dots , x^{(M)} \in \mathbb{R}^n$
, their convex hull is the set of all $x$ that can be written as a convex combination of these points.

1. Use the Frank-Wolfe algorithm to compute the projection of a query point $q$ onto the convex hull of the given $x^{(1)},\dots , x^{(M)} \in \mathbb{R}^n$. That is, you should solve the following optimization problem.
\begin{align}
\min_{\lambda \in \mathbb{R}^n} \frac{1}{2}\Vert q - \sum_{m=1}^M \lambda_m x^{(m)} \Vert_2^2
\end{align}
such that
\begin{align}
\sum_{m=1}^M \lambda_m &= 1\\
\lambda &\succeq 0
\end{align}
Your Python function should take as input the $x$’s, the query point $q$, an initial value for $\lambda$ that satisfies the constraints, and the tolerance for the Frank-Wolfe convergence condition and return the best function value found during the iterative procedure.

**Conditional Gradients (Franke-Wolfe)**

Formally
\begin{align}
f_0(x) \approx f_0(x_t) + \nabla f_0(x_t)^T(x-x_t).
\end{align}
Compute
\begin{align}
s_t \in \text{argmin}_{x\in C} \nabla f_0(x_t)^T x
\end{align}
Set
\begin{align}
x_{t+1} = x_t + \gamma_t(s_t - x_t)
\end{align}
The algorithm is guaranteed to converge with a diminishing step size choice like
\begin{align}
\gamma_{t} = \frac{2}{2+t}
\end{align}

So
\begin{align}
f(\lambda) = &\frac{1}{2}\Vert q - \sum_{m=1}^M \lambda_m x^{(m)} \Vert_2^2\\
&= \frac{1}{2} \sum_{i=1}^{n} \left(q_i - \sum_{m=1}^M \lambda_mx^{(m)}_i\right)^2\\
\nabla f(\lambda)_j &=  - \sum_{i = 1}^n x_i^{(j)}\left(q_i - \sum_{m=1}^M \lambda_m x_i^{(m)}\right)  
\end{align} \\




In [ ]:
def get_gradient(lambd_a):
  gradient = np.zeros([M])
  for j in range(gradient.shape[0]):
    sum_n = 0
    for i in range(n):
      sum_m = 0
      for m in range(gradient.shape[0]):

        sum_m += lambd_a[m]*x[m][i]

      sum_n += x[j][i] * (q[i] - sum_m)

    gradient[j] = -sum_n

  return gradient

To solve the per-iteration subproblems, use scipy
\begin{align}
s_t &= \text{argmin}_{\substack{\lambda \in [0,1]^M\\\sum_{i=1}^M \lambda_i = 1}}\nabla f(\lambda^{\text{old}})^T \lambda
\end{align}

In [ ]:
def s_finder(lambd_a):
  
  gradient = get_gradient(lambd_a)
  M = gradient.shape[0]
  l = 0
  u = 1
  A_eq = np.ones([1,M])
  b_eq = np.ones([1,1])
  s_t = optimize.linprog(c = gradient, bounds = [l,u], A_eq = A_eq,b_eq = b_eq)
  
  return s_t.x

And then
\begin{align}\lambda^{\text{new}} &= \lambda^{\text{old}} + \gamma (s_t - \lambda^{\text{old}})
\end{align}

In [ ]:
def Frank_Wolfe(max_iterations,initial_lambd_a,epsilon):
  lambd_a = initial_lambd_a

  for iteration in range(1,max_iterations):

    gamma = 2 / (2 + iteration)

    s_t = s_finder(lambd_a)
    step = gamma*(s_t - lambd_a)

    if sum(abs(step)) < epsilon: #Stopping Criteria
      break

    lambd_a = lambd_a + step


  return lambd_a

In [ ]:
import numpy as np
from scipy import optimize


x = np.array([[-1,-1],[-1,1],[1,-1],[1,1]])
q = np.array([1.2,0.2])

n = 2
M = 4

max_iterations = 1000
initial_lambd_a = np.zeros(x.shape[0])
epsilon = 0.00001


final_lambda = Frank_Wolfe(max_iterations,initial_lambd_a,epsilon)
print(final_lambda)
point = np.zeros(n)
for m in range(M):
  point += final_lambda[m]*x[m]
print(point)

[2.76927988e-10 2.76408737e-10 4.00799284e-01 5.99198718e-01]
[0.999998   0.19839943]
